In [1]:
import configparser
import pandas as pd
import datetime
import json
import os

In [2]:
from pythonFitbitMaster import fitbit
from pythonFitbitMaster import gather_keys_oauth2 as Oauth2

In [14]:
# emplacements des datasets, et nomenclature
datasets_location = "/media/mdercq/sedentaire/Nomade/15_datasets/"
fitbit_folder = "fitbit_data"
intradayHR_folder = "intraday_heart"
heartRateZones_folder = "heartrate_zones"
sleep_status_folder = 'sleep_status'

restingHR_file = "restingHR.txt"
totalMinutesAsleep = 'totalMinutesAsleep'
totalTimeInBed = 'totalTimeInBed'
sleep_efficiency = 'sleepEfficiency'
sleep_timings = 'sleepTimings'

myfitbit_configfile = 'myfitbit.ini'

In [15]:
# settings
config = configparser.ConfigParser()
config.read(myfitbit_configfile)
last_day = config['date_time']['last_date']

In [5]:
# connection to server
server = Oauth2.OAuth2Server(config['fitbit_auth']["client_id"],
                             config['fitbit_auth']["client_secret"])
server.browser_authorize()
ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])
auth2_client = fitbit.Fitbit(config['fitbit_auth']["client_id"],
                             config['fitbit_auth']["client_secret"],
                             access_token=ACCESS_TOKEN,
                             refresh_token=REFRESH_TOKEN,
                             oauth2=True)

[23/Sep/2019:22:33:47] ENGINE Listening for SIGTERM.
[23/Sep/2019:22:33:47] ENGINE Listening for SIGHUP.
[23/Sep/2019:22:33:47] ENGINE Listening for SIGUSR1.
[23/Sep/2019:22:33:47] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[23/Sep/2019:22:33:47] ENGINE Started monitor thread 'Autoreloader'.
[23/Sep/2019:22:33:47] ENGINE Serving on http://127.0.0.1:8080
[23/Sep/2019:22:33:47] ENGINE Bus STARTED


127.0.0.1 - - [23/Sep/2019:22:33:49] "GET /?code=f8a2edf3e94fbc43dfa97b846aded09c64928f78&state=NQQwNFrRSkynobw92p4m3JQRyxHVp0 HTTP/1.1" 200 122 "" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0"
127.0.0.1 - - [23/Sep/2019:22:33:49] "GET /favicon.ico HTTP/1.1" 200 1406 "" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0"


[23/Sep/2019:22:33:50] ENGINE Bus STOPPING
[23/Sep/2019:22:33:50] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[23/Sep/2019:22:33:50] ENGINE Stopped thread 'Autoreloader'.
[23/Sep/2019:22:33:50] ENGINE Bus STOPPED
[23/Sep/2019:22:33:50] ENGINE Bus EXITING
[23/Sep/2019:22:33:50] ENGINE Bus EXITED
[23/Sep/2019:22:33:50] ENGINE Waiting for child threads to terminate...


In [6]:
config['fitbit_auth']['access_token'] = ACCESS_TOKEN
config['fitbit_auth']['refresh_token'] = REFRESH_TOKEN

In [20]:
def collect_heart_rate(day_to_collect):
    # get the Heart rate intraday data
    fit_statsHR = auth2_client.intraday_time_series('activities/heart', base_date=day_to_collect, detail_level='1sec')
    # collecting the time series
    fullpath = os.path.join(datasets_location, fitbit_folder, intradayHR_folder, day_to_collect + '.csv')
    intra_day_HR = pd.DataFrame(fit_statsHR['activities-heart-intraday']['dataset'])
    intra_day_HR.to_csv(fullpath, header=True, index=True, sep='\t', na_rep='n/a')
    
    # collecting the summary
    fullpath = os.path.join(datasets_location, fitbit_folder, heartRateZones_folder, day_to_collect + '.csv')
    heartRateZones_data = pd.DataFrame(fit_statsHR['activities-heart'][0]['value']['heartRateZones'])
    heartRateZones_data.to_csv(fullpath, header=True, index=True, sep='\t', na_rep='n/a')
    
    # collecting the resting value
    resting_value = str(fit_statsHR['activities-heart'][0]['value']['restingHeartRate'])
    new_line = '{}\t{}\n'.format(day_to_collect, resting_value)
    with open(os.path.join(datasets_location, fitbit_folder, restingHR_file), 'a') as file:
        file.write(new_line)

In [21]:
def collect_sleep_data(day_to_collect):
    # collect sleep data
    fit_statsSl = auth2_client.sleep(date=day_to_collect)
    # record some summaries
    with open(os.path.join(datasets_location, fitbit_folder, totalMinutesAsleep), 'a') as file:
        file.write('{}\t{}\n'.format(day_to_collect,
                                     fit_statsSl['summary']['totalMinutesAsleep']))
    with open(os.path.join(datasets_location, fitbit_folder, totalTimeInBed), 'a') as file:
        file.write('{}\t{}\n'.format(day_to_collect,
                                     fit_statsSl['summary']['totalTimeInBed']))
    with open(os.path.join(datasets_location, fitbit_folder, sleep_efficiency), 'a') as file:
        file.write('{}\t{}\n'.format(day_to_collect,
                                     fit_statsSl['sleep'][0]['efficiency']))
    with open(os.path.join(datasets_location, fitbit_folder, sleep_timings), 'a') as file:
        file.write('{}\t{}\t{}\n'.format(day_to_collect,
                                         fit_statsSl['sleep'][0]['startTime'],
                                         fit_statsSl['sleep'][0]['endTime']))
    # record the time series
    fullpath = os.path.join(datasets_location, fitbit_folder, sleep_status_folder, day_to_collect + '.csv')
    sleep_status = pd.DataFrame(fit_statsSl['sleep'][0]['minuteData'])
    sleep_status['Interpreted'] = sleep_status['value'].map({'2':'Awake','3':'Very Awake','1':'Asleep'})
    sleep_status.to_csv(fullpath, header=True, index=True, sep='\t', na_rep='n/a')

In [22]:
with open(myfitbit_configfile, 'w') as configfile:
    config.write(configfile)

In [23]:
initial_day = last_day
initial_day_dt = datetime.datetime.strptime(initial_day, '%Y-%m-%d')
for i in range(149):
    date_to_process = initial_day_dt + datetime.timedelta(days=i)
    collect_heart_rate(date_to_process.strftime('%Y-%m-%d'))
    collect_sleep_data(date_to_process.strftime('%Y-%m-%d'))

In [12]:
config['date_time']['last_date'] = str(date_to_process.timetstr('%Y-%m-%d'))
print(config['date_time']['last_date'])
with open(myfitbit_configfile, 'w') as configfile:
    config.write(configfile)

NameError: name 'date_to_process' is not defined